# XGBOOST for classification and Hyperparameter using Optuna

In [1]:
#importing libraries

import optuna
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import sklearn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [2]:
#loading data
breast_cancer = load_breast_cancer()

In [3]:
#Describe the dataset
print(breast_cancer.DESCR)

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 569

    :Number of Attributes: 30 numeric, predictive attributes and the class

    :Attribute Information:
        - radius (mean of distances from center to points on the perimeter)
        - texture (standard deviation of gray-scale values)
        - perimeter
        - area
        - smoothness (local variation in radius lengths)
        - compactness (perimeter^2 / area - 1.0)
        - concavity (severity of concave portions of the contour)
        - concave points (number of concave portions of the contour)
        - symmetry
        - fractal dimension ("coastline approximation" - 1)

        The mean, standard error, and "worst" or largest (mean of the three
        worst/largest values) of these features were computed for each image,
        resulting in 30 features.  For instance, field 0 is Mean Radi

In [4]:
#print the features of the dataset
print(breast_cancer.feature_names)

['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']


In [5]:
#print the target of the dataset
print(breast_cancer.target_names)

['malignant' 'benign']


In [6]:
X = breast_cancer.data
y = breast_cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
#printing the shapes of the train and test sets
print("X_train shape: ", X_train.shape)
print("X_test shape: ", X_test.shape)
print("y_train shape: ", y_train.shape)
print("y_test shape: ", y_test.shape)

X_train shape:  (455, 30)
X_test shape:  (114, 30)
y_train shape:  (455,)
y_test shape:  (114,)


In [8]:
#fit a XGBoost Classifier with default parameters
clf_xgb = xgb.XGBClassifier()
clf_xgb.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [9]:
#predict the test set
y_pred = clf_xgb.predict(X_test)

#calculate the performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print("Task 2: Default XGBoost Classifier Performance:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1-Score: {f1:.2f}")
print(f"ROC-AUC: {roc_auc:.2f}")

Task 2: Default XGBoost Classifier Performance:
Accuracy: 0.96
Precision: 0.96
Recall: 0.97
F1-Score: 0.97
ROC-AUC: 0.95


# Hyperparameter using Optuna

In [10]:
#defining the objective function

def objective(trial):

    params = {
        "verbosity": 0,
        "objective": "binary:logistic",
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 10, 100, step=1),
        "subsample": trial.suggest_float("subsample", 0.6, 0.9),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracy = sklearn.metrics.accuracy_score(y_test, y_pred)
    return accuracy

In [11]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

best_params = study.best_params
best_model = xgb.XGBClassifier(**best_params)

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
best_accuracy = accuracy_score(y_test, y_pred)
best_precision = precision_score(y_test, y_pred)
best_recall = recall_score(y_test, y_pred)
best_f1 = f1_score(y_test, y_pred)
best_roc_auc = roc_auc_score(y_test, y_pred)

print("TBest Hyperparameters from Optuna:")
print(best_params)

print("Best XGBoost Classifier Performance:")
print(f"Accuracy: {best_accuracy:.2f}")
print(f"Precision: {best_precision:.2f}")
print(f"Recall: {best_recall:.2f}")
print(f"F1-Score: {best_f1:.2f}")
print(f"ROC-AUC: {best_roc_auc:.2f}")

[I 2023-10-04 18:59:34,411] A new study created in memory with name: no-name-ae87f0dd-3cca-41fe-9260-2d6d02167942
[I 2023-10-04 18:59:50,967] Trial 0 finished with value: 0.9649122807017544 and parameters: {'booster': 'gbtree', 'max_depth': 7, 'learning_rate': 0.1919154179947743, 'n_estimators': 33, 'subsample': 0.6658746970180826, 'colsample_bytree': 0.8661643381673766}. Best is trial 0 with value: 0.9649122807017544.
[I 2023-10-04 18:59:56,167] Trial 1 finished with value: 0.9736842105263158 and parameters: {'booster': 'gbtree', 'max_depth': 8, 'learning_rate': 0.23314258023458886, 'n_estimators': 46, 'subsample': 0.7914360382192613, 'colsample_bytree': 0.8693677274293565}. Best is trial 1 with value: 0.9736842105263158.
[I 2023-10-04 18:59:57,296] Trial 2 finished with value: 0.956140350877193 and parameters: {'booster': 'gblinear', 'max_depth': 5, 'learning_rate': 0.14472915075332654, 'n_estimators': 69, 'subsample': 0.6797303742139931, 'colsample_bytree': 0.8781268294023192}. Best

[19:16:56] WARNING: ../src/learner.cc:767: 
Parameters: { "colsample_bytree", "max_depth", "subsample" } are not used.

TBest Hyperparameters from Optuna:
{'booster': 'gblinear', 'max_depth': 7, 'learning_rate': 0.2112313669279117, 'n_estimators': 91, 'subsample': 0.8484044756625635, 'colsample_bytree': 0.672932967850814}
Best XGBoost Classifier Performance:
Accuracy: 0.97
Precision: 0.99
Recall: 0.97
F1-Score: 0.98
ROC-AUC: 0.97
